In [3]:
import os
import sys
import re
import time
import random
import warnings
import collections
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns

sys.path.append('../../src')
import cb_utils

sns.set(style="darkgrid")
pd.options.display.max_columns = 500

%load_ext autoreload
%autoreload 2

# Milliman PAC File ETL
1) log into SFTP site: https://secureftp.milliman.com  
2) In the "from milliman" folder, go to the latest directory  
     - In this case, the latest directory is "20221216"  
3) Make sure you have access to the following folder: \\Main Street Health\MSH_Strategy_Group - ACO Analyses\Raw MIlliman Flat FIles\  
     - create folder with same datestamp as file from milliman, in this case "20221216"  
4) Download the files from the SFTP site  
     - in this case there are 3  
5) copy the data disctionary to the folder on the network drive.  
6) download the prospective and retrospective CSV files into database tables in both the MSH and CB datagrip environments (Alan has a place he puts them)  
7) delete the last 30 columns from these database tables (we will create a "pivot file" with these 30 columns):  
8) with these 30 columns, run Alan's "pivot code" to create these files:  
     - strategic_milliman_prosp_20221128_pivot
     -  strategic_milliman_retro_20221128_pivot
9) copy 4 files to the "20221216" folder  
     - prospective flat file (without the 30 MA columns)
     - retrospective flat file (without the 30 MA columns)
     - prospective MA pivot file
     - retrospective MA pivor file
10) email Ali and Ben that process is complete with the following summaries that you can pull via queries and paste in excel into the same folder.  

In [24]:
data_dir = '/Users/bp/workspace/msh/milliman_data/'
current_folder = '20230202'
old_pro_file_name = 'Prospective Flat File.txt'
retro_file_name = 'Milliman ACO Builder - MSSP Retrospective Explorer v2024.1.2.txt'

### Pro load

In [30]:
data_dir = '/Users/bp/workspace/msh/milliman_data/'
current_folder = '20230317'
pro_file_name = 'Milliman ACO Builder - MSSP Prospective Explorer v2024.1.2.txt'
retro_file_name = 'Milliman ACO Builder - MSSP Retrospective Explorer v2024.1.2.txt'

### Pro load

In [31]:
pro_df = pd.read_csv(f'{data_dir}/{current_folder}/{pro_file_name}',  encoding='latin1', low_memory=False, delimiter='|')
# pro_df = cb_utils.df_format_columns(pro_df)
print(f'{pro_df.shape[0]} rows, {pro_df.shape[1]} columns')
pro_df.head()

613556 rows, 95 columns


,PAC_ID,DeID_TIN,year,program_type,PRVDR_NAME,ACO_Name,PRVDR_Address,PRVDR_ZIP,PRVDR_ST,PRVDR_State,PRVDR_MSACode,PRVDR_ST_MSA,PRVDR_MSAName,Rural_Ind,Spec_Count,NP_Count,CNS_Count,PCP_Count,PA_Count,Other_Count,unique_members,MMs_ESRD,MMs_Dis,MMs_Ad,MMs_AND,MMs_Total,RS_ESRD,RS_Dis,RS_Ad,RS_AND,Risk_Composite,Paid_ESRD,Paid_Dis,Paid_Ad,Paid_AND,Paid_Composite,Reg_Ret_Exp_ESRD,Reg_Ret_Exp_Dis,Reg_Ret_Exp_Ad,Reg_Ret_Exp_AND,Reg_Ret_Exp_Composite,Regional_Efficiency_Ret,Reg_Pro_Exp_ESRD,Reg_Pro_Exp_Dis,Reg_Pro_Exp_Ad,Reg_Pro_Exp_AND,Reg_Pro_Exp_Composite,Regional_Efficiency_Pro,ip_cost,ip_admit,readmissions,snf_cost,snf_admit,er_cost,er_util,hh_cost,hh_util,pb_cost,primary_care_cost,op_cost,hp_cost,dme_cost,ip_cost_keep,snf_cost_keep,er_cost_keep,hh_cost_keep,pb_cost_keep,primary_care_cost_keep,op_cost_keep,hp_cost_keep,dme_cost_keep,awv,awv_keep,er_needed_unavoid,er_need_avoid,er_emerg_prim_care_treat,er_non_emerg,er_injury,er_psych,er_alcohol,er_drug,er_unclassified,unique_potl_members,mms_potl_esrd,mms_potl_dis,mms_potl_ad,mms_potl_and,mms_pcp_esrd,mms_pcp_dis,mms_pcp_ad,mms_pcp_and,mms_spc_esrd,mms_spc_dis,mms_spc_ad,mms_spc_and
0,42100091,441806,2017,MSSP Prospective,UNITED NEIGHBORHOOD HEALTH SERVICES,Not Available,"2711 FOSTER AVE, NASHVILLE, TN 372105307",37210,TN,Tennessee,34980.0,TN-34980,"Nashville-Davidson--Murfreesboro--Franklin, TN",FQHC,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,24.00,5.16,0.00,29.16,NaN,0.298518,0.548212,NaN,0.358556,NaN,14.623333,21.531008,NaN,15.845679,0.0000,883.681667,1439.504167,0.000000,1195.101387,0.044416,0.000000,842.744167,1357.345833,0.000000,1134.709223,0.046779,NaN,0.0,0.0,NaN,0.0,NaN,0.0,NaN,0.0,206.387755,263.18,251.551020,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.00,0.000000,NaN,NaN,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,NaN,0.0,64.0,32.0,NaN,0.0,24.0,NaN,0.0,0.0,0.0,0.0,0.0
1,42100091,441806,2018,MSSP Prospective,UNITED NEIGHBORHOOD HEALTH SERVICES,Not Available,"2711 FOSTER AVE, NASHVILLE, TN 372105307",37210,TN,Tennessee,34980.0,TN-34980,"Nashville-Davidson--Murfreesboro--Franklin, TN",FQHC,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,12.00,0.00,0.00,12.00,NaN,1.311401,NaN,NaN,1.311401,NaN,188.360833,NaN,NaN,188.360833,0.0000,893.815833,0.000000,0.000000,893.815833,0.160697,0.000000,847.528333,0.000000,0.000000,847.528333,0.169473,NaN,0.0,0.0,NaN,0.0,762.469388,NaN,NaN,0.0,734.938776,938.40,1525.387755,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.00,0.000000,NaN,NaN,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,NaN,0.0,0.0,NaN,NaN,0.0,48.0,0.0,12.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0
2,42100091,441820,2016,MSSP Prospective,CAYCE MEDICAL AND DENTAL CENTER,Not Available,"2711 FOSTER AVE, NASHVILLE, TN 372105307",37210,TN,Tennessee,34980.0,TN-34980,"Nashville-Davidson--Murfreesboro--Franklin, TN",FQHC,0.0,NaN,0.0,NaN,0.0,0.0,66.0,12.0,462.96,141.96,128.04,744.96,0.769280,0.836305,0.982770,1.148692,0.895635,2593.068333,1099.708290,1239.594393,580.368479,1061.158948,6377.6700,830.514167,1338.752500,752.874167,1016.156470,1.166071,6321.826667,769.571667,1280.828333,738.190833,964.666636,1.228311,611888.73469,42.0,11.0,31852.989796,NaN,27659.234694,70.0,45804.724490,16.0,182084.663270,32096.39,123802.561220,19396.27551,14286.581633,0.0,0.0,0.0,0.0,0.0,14458.19,11754.173469,0.0,0.0,NaN,0.0,16.962230,NaN,15.965212,11.702848,13.0,NaN,NaN,0.0,NaN,104.0,12.0,754.0,217.0,193.0,12.0,463.0,142.0,128.0,0.0,0.0,0.0,0.0
3,42100091,441820,2017,MSSP Prospective,CAYCE MEDICAL AND DENTAL CENTER,Not Available,"2711 FOSTER AVE, NASHVILLE, TN 372105307",37210,TN,Tennessee,34980.0,TN-34980,"Nashville-Davidson--Murfreesboro--Franklin, TN",FQHC,0.0,NaN,0.0,NaN,0.0,0.0,76.0,24.0,545.04,147.00,132.96,849.00,1.035063,0.961642,0.584747,0.596986,0.913241,3091.865000,615.993285,339.876463,211.125451,574.768928,6271.6525,893.760833,1425.158333,783.472500,1043.288082,0.603233,6341.302500,851.656667,1345.968333,762.689167,1006.151615,0.625498,142858.23469,23.0,NaN,17376.489796,NaN,51280.510204,104.0,53944.897959,20.0,117509.857140,42648.14,

In [32]:
pro_df.to_csv('pro.csv',index=False)

In [6]:
conn = cb_utils.get_engine()
pro_df.to_sql(f'milliman_pro_{current_folder}', conn, schema='strategic', index=False, method='multi', chunksize=1000) # , if_exists='replace'

613556

### Retro load

In [9]:
ret_df = pd.read_csv(f'{data_dir}/{current_folder}/{retro_file_name}',  encoding='latin1', low_memory=False, delimiter='|')
ret_df = cb_utils.df_format_columns(ret_df)
print(f'{ret_df.shape[0]} rows, {ret_df.shape[1]} columns')
ret_df.head()

594838 rows, 95 columns


,pac_id,deid_tin,year,program_type,prvdr_name,aco_name,prvdr_address,prvdr_zip,prvdr_st,prvdr_state,prvdr_msacode,prvdr_st_msa,prvdr_msaname,rural_ind,spec_count,np_count,cns_count,pcp_count,pa_count,other_count,unique_members,mms_esrd,mms_dis,mms_ad,mms_and,mms_total,rs_esrd,rs_dis,rs_ad,rs_and,risk_composite,paid_esrd,paid_dis,paid_ad,paid_and,paid_composite,reg_ret_exp_esrd,reg_ret_exp_dis,reg_ret_exp_ad,reg_ret_exp_and,reg_ret_exp_composite,regional_efficiency_ret,reg_pro_exp_esrd,reg_pro_exp_dis,reg_pro_exp_ad,reg_pro_exp_and,reg_pro_exp_composite,regional_efficiency_pro,ip_cost,ip_admit,readmissions,snf_cost,snf_admit,er_cost,er_util,hh_cost,hh_util,pb_cost,primary_care_cost,op_cost,hp_cost,dme_cost,ip_cost_keep,snf_cost_keep,er_cost_keep,hh_cost_keep,pb_cost_keep,primary_care_cost_keep,op_cost_keep,hp_cost_keep,dme_cost_keep,awv,awv_keep,er_needed_unavoid,er_need_avoid,er_emerg_prim_care_treat,er_non_emerg,er_injury,er_psych,er_alcohol,er_drug,er_unclassified,unique_potl_members,mms_potl_esrd,mms_potl_dis,mms_potl_ad,mms_potl_and,mms_pcp_esrd,mms_pcp_dis,mms_pcp_ad,mms_pcp_and,mms_spc_esrd,mms_spc_dis,mms_spc_ad,mms_spc_and
0,42100091,441806,2016,MSSP Retrospective,UNITED NEIGHBORHOOD HEALTH SERVICES,Not Available,"2711 FOSTER AVE, NASHVILLE, TN 372105307",37210,TN,Tennessee,34980.0,TN-34980,"Nashville-Davidson--Murfreesboro--Franklin, TN",FQHC,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.00,12.00,12.00,0.00,24.00,NaN,0.516226,0.442639,NaN,0.501028,NaN,48.431667,12.606667,NaN,30.519167,0.000,827.329167,1343.819167,0.000000,1019.818775,0.059729,0,0,0,0,0,0,NaN,0.0,0.0,NaN,0.0,66.581633,NaN,NaN,0.0,344.938776,394.77,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,263.18,NaN,NaN,NaN,0.0,0.0,0.00000,0.0,0.000000,0.000000,NaN,0.0,0.0,0.0,0.0,NaN,0.0,27.0,36.0,0.0,0.0,12.0,12.0,0.0,0.0,0.0,0.0,0.0
1,42100091,441806,2017,MSSP Retrospective,UNITED NEIGHBORHOOD HEALTH SERVICES,Not Available,"2711 FOSTER AVE, NASHVILLE, TN 372105307",37210,TN,Tennessee,34980.0,TN-34980,"Nashville-Davidson--Murfreesboro--Franklin, TN",FQHC,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.00,14.04,0.00,0.00,14.04,NaN,0.355729,NaN,NaN,0.355729,NaN,1030.366097,NaN,NaN,1030.366097,0.000,906.930000,0.000000,0.000000,904.346154,3.193730,0,0,0,0,0,0,8690.744898,NaN,0.0,NaN,0.0,1801.000000,NaN,NaN,0.0,3364.193878,539.90,2411.397959,NaN,NaN,0.0,NaN,0.0,NaN,0.0,408.31,331.948980,NaN,NaN,0.0,0.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0,0.0,NaN,NaN,0.0,36.0,0.0,12.0,0.0,14.0,0.0,0.0,0.0,0.0,0.0,0.0
2,42100091,441820,2016,MSSP Retrospective,CAYCE MEDICAL AND DENTAL CENTER,Not Available,"2711 FOSTER AVE, NASHVILLE, TN 372105307",37210,TN,Tennessee,34980.0,TN-34980,"Nashville-Davidson--Murfreesboro--Franklin, TN",FQHC,0.0,NaN,0.0,NaN,0.0,0.0,77.0,24.00,543.00,141.00,75.00,783.00,0.719069,0.756120,0.652561,0.473906,0.731750,2543.5575,512.411842,310.973830,131.047333,501.865734,6377.670,822.570833,1338.716667,755.004167,1043.354762,0.657344,0,0,0,0,0,0,107783.765310,22.0,NaN,1099.948980,NaN,33406.204082,98.0,22223.397959,NaN,149008.051020,39806.45,NaN,NaN,5115.204082,0.0,0.0,0.0,0.0,0.0,28658.09,NaN,NaN,0.0,NaN,0.0,11.49377,NaN,18.714193,22.897583,21.0,NaN,NaN,0.0,NaN,121.0,48.0,877.0,184.0,159.0,24.0,543.0,141.0,75.0,0.0,0.0,0.0,0.0
3,42100091,441820,2017,MSSP Retrospective,CAYCE MEDICAL AND DENTAL CENTER,Not Available,"2711 FOSTER AVE, NASHVILLE, TN 372105307",37210,TN,Tennessee,34980.0,TN-34980,"Nashville-Davidson--Murfreesboro--Franklin, TN",FQHC,0.0,NaN,0.0,NaN,0.0,0.0,57.0,30.96,350.04,111.96,96.96,589.92,1.092130,0.921279,0.560636,0.573331,0.915871,4844.2500,418.183465,373.066452,373.018152,634.484811,5998.335,894.505833,1424.335000,787.450833,1156.503440,0.599390,0,0,0,0,0,0,125097.704080,16.0,NaN,9483.959184,NaN,26573.489796,51.0,40048.448980,14.0,88227.224490,33117.03,143017.663270,NaN,4714.826531,0.0,0.0,0.0,0.0,0.0,25689.14,20884.785714,NaN,0.0,NaN,0.0,NaN,NaN,11.570058,NaN,NaN,NaN,0.0,0.0,NaN,95.0,31.0,690.0,151.0,158.0,31.0,350.0,112.0,97.0,0.0,0.0,0.0,0.0
4,42100091,441820,2018,MSSP Retrospective,CAYCE MEDICAL 

In [33]:

ret_df.to_csv('ret.csv',index=False)

In [10]:
conn = cb_utils.get_engine()
ret_df.to_sql(f'milliman_ret_{current_folder}', conn, schema='strategic', index=False, method='multi', chunksize=1000) # if_exists='replace', 

594838

### Pivots

In [11]:
def pivot_ma_plan_enrollment(df):
    dfs = []
    for i in range(1, 16):
        name, enrollment = f'ma_plan_{i}_name', f'ma_plan_{i}_enrollment'
        plan = df[['deid_tin', 'pac_id', 'year', name, enrollment]]
        plan = plan.loc[~(plan[name].isna()) & ~(plan[enrollment].isna())]
        plan.columns = ['deid_tin', 'pac_id', 'year', 'ma_plan_name', 'ma_enrollment']
        dfs.append(plan)
    return pd.concat(dfs)

In [12]:
ret_pivot_df = pivot_ma_plan_enrollment(ret_df)

KeyError: "['ma_plan_1_name', 'ma_plan_1_enrollment'] not in index"

In [ ]:
ret_pivot_df.head()

In [13]:
pro_pivot_df = pivot_ma_plan_enrollment(pro_df)

KeyError: "['ma_plan_1_name', 'ma_plan_1_enrollment'] not in index"

In [ ]:
pro_pivot_df.head()

### Summaries 
### Prospective

In [ ]:
s1 = ret_df[['year', 'unique_members']].groupby('year', as_index=False).sum()
s1.head()

### Retro

In [ ]:
s2 = pro_df[['year', 'unique_members']].groupby('year', as_index=False).sum()
s2.head()

### Prospective

In [ ]:
s3 = pro_pivot_df[['year', 'ma_plan_name', 'ma_enrollment']].groupby(['year', 'ma_plan_name'], as_index=False).sum().sort_values('ma_enrollment', ascending=False)
s3.head()

### Retro

In [ ]:
s4 = ret_pivot_df[['year', 'ma_plan_name', 'ma_enrollment']].groupby(['year', 'ma_plan_name'], as_index=False).sum().sort_values('ma_enrollment', ascending=False)
s4.head()

### Outputs

In [ ]:
ret_df.loc[:, ret_df.columns[:-30]].to_csv(f'{data_dir}/{current_folder}/ours_milliman_ret_{current_folder}.csv', index=False)

In [ ]:
pro_df.loc[:, pro_df.columns[:-30]].to_csv(f'{data_dir}/{current_folder}/ours_milliman_pro_{current_folder}.csv', index=False)

In [ ]:
pro_pivot_df.to_csv(f'{data_dir}/{current_folder}/pro_pivot.csv', index=False)
ret_pivot_df.to_csv(f'{data_dir}/{current_folder}/ret_pivot.csv', index=False)

In [ ]:
with pd.ExcelWriter(f'{data_dir}/{current_folder}/summary.xlsx') as writer:
    s1.to_excel(writer, sheet_name='retrospective', index=False)
    s2.to_excel(writer, sheet_name='prospective', index=False)
    s3.to_excel(writer, sheet_name='prospective_pivot', index=False)
    s4.to_excel(writer, sheet_name='retrospective_pivot', index=False)